In [ ]:
import datetime
import numpy as np
import pandas as pd
from pathlib import Path

from dash import Dash, dcc, html
# import dash_table
# import dash.dash_core_components as dcc
# import dash.dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.dependencies import Input, Output, State

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'newplot',
    'height': 1000,
    'width': 1000,
    'scale':1 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

airport_list = ('LEBL','LPPT','LFPO','LFPG','EGLL','LPPR','EDDF','EBBR','EHAM','LEBB','LIRF',
                'EDDM','EGKK','LEVC','LSGG','LECO','LIMC','EIDW','LFML','LEZL','LOWW','LIPZ',
                'LIPE','LFLL','EDDL','EDDB','LTFM','LFMN','LFRS','LROP','EDDP','LGAV','EDDH',
                'LHBP','EKCH','EGCC','ELLX','LKPR','LIRN','LBSF','EPWA')

import sys
sys.path.append('./rtaUtils')

from rtaUtils import data_loading, common, sort_vectors, paths

from tqdm import tqdm

## Inicialización

In [ ]:
# Estilos de los ejemplos de Dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
# Carga de datos
data = pd.read_csv('./RTAutils/sort_stats.csv').drop_duplicates(subset='fpId', keep='last')
data['flightDate'] = pd.to_datetime(data.flightDate)
df = None

## Estructura

In [ ]:
# Declaración de elementos y construcción del layout
style_dd = dict(minWidth = 250, width='50%', maxWidth=400, fontSize='95%', )
style_dp = dict(marginLeft=20)
style_lb = {'margin':10, 'maxWidth':200 , 'display':'inline-block'}

dates = common.get_dates_between(date_start = data.flightDate.min().strftime('%Y-%m-%d'), 
                                 date_end   = data.flightDate.max().strftime('%Y-%m-%d'))

dates = [f'2022-{m:0>2}-{d:0>2}' for m in range(1,10) for d in range(1,32) 
         if f'2022-{m:0>2}-{d:0>2}' not in data.flightDate.dt.date.astype(str).values]

# Selector de fecha
sel_fecha = dcc.DatePickerSingle(
    id = 'sel_fecha',
    placeholder = 'Selecciona...',
    date=data.flightDate.min(),
    initial_visible_month=data.flightDate.min(),
    min_date_allowed = data.flightDate.min(),
    max_date_allowed = data.flightDate.max(),
    display_format='DD/MM/YYYY',
    disabled_days=dates,
    style=style_dp
)
# Lista de opciones de aeropuerto de origen
dd_origen = dcc.Dropdown(
    id = 'dd_origen',
    options = [dict(label=x, value=x) for x in sorted(airport_list)],
    placeholder='Selecciona...',
    multi=True,
    
#     style = style_dp,
)

# Lista de opciones de trayectorias disponibles
# - Si no se ha filtrado, se muestran todas las del día seleccionado
# - Si se ha filtrado, se muestran las trayectorias que cumplen todas las condiciones
dd_trayectoria = dcc.Dropdown(
    id = 'dd_trayectoria',
    placeholder='Selecciona...',
    multi=False,
#     style = style_dp,
)


distanceSlider = dcc.RangeSlider(0,  100, step=25, value=[0,100], 
                                 id='my-slider', tooltip={"placement": "top", "always_visible": False},
                                 allowCross=False, pushable=5)

app.layout = html.Div(children=[
    html.Div(style=dict(width='100%',  borderWidth=1, display='flex', 
                        borderStyle='solid', borderRadius=10), children=[
        html.Div(style=dict(width='30%', padding=10), children=[
            html.Div(style=dict(display='flex'), children=[
                html.Label(['Trayectoria', dd_trayectoria],style=dict(width='75%')),
                html.Label(['Mostrar:', 
                dcc.Dropdown(id='color_scale_dd', value='ordenInicial', clearable=False,options = {
                    'ordenInicial':'Orden inicial',
                    'track':'Track',
                })],style=dict(width='25%', marginLeft=10))]),
            html.Label(['Distancia', distanceSlider])
        ]),
        html.Div(style=dict(width='15%', padding=10, marginLeft=20), children=[
            #html.Div(style=dict(display='flex', alignItems='center'), children=[
                html.Label(['Fecha', sel_fecha]),
                #html.Button(children=['Filtrar'], id='btn_filtrar', style=dict(marginLeft='auto'))]),
                html.Label(['Aeropuerto de origen', dd_origen])
        ]),
        html.Div(id='trayectory_description', style=dict(width='40%', padding=5))
    ]),
    html.Div(style=dict(width='100%', display='flex'), children=[
            html.Div(id='inicial', style=dict(width='50%'),
                     children=[dcc.Graph(id='grafico_inicial', style=dict(width='100%'))]),
            html.Div(id='reordenado', style=dict(width='50%'),
                     children=[dcc.Graph(id='grafico_reordenado', style=dict(width='100%'))])
    ])
])

## Callbacks

In [ ]:
@app.callback(
    [Output(component_id='dd_trayectoria', component_property='options')],
    # [Input(component_id='btn_filtrar', component_property='n_clicks')],
    [Input(component_id='sel_fecha', component_property='date'),
     Input(component_id='dd_origen', component_property='value')]
)
def filtrar_trayectorias(fecha, origen):
#     global df
    
    df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
#     if clicks == None:
#         return ([{'label':x, 'value':x} for x in df.fpId.unique()],)
    if origen:
        df = df[df.aerodromeOfDeparture.isin(origen)]   
    labels = data.groupby(['fpId','rotation','ratio','oscillation']).sum().reset_index()[['fpId','rotation','ratio','oscillation']]
    labels2 = df.groupby(['fpId','aerodromeOfDeparture']).sum().reset_index()[['fpId','aerodromeOfDeparture']]
    labels = pd.merge(labels, labels2, on = 'fpId')
    
    return ([{'label': f'{x[1].aerodromeOfDeparture}    {x[1].fpId}    ({x[1].rotation:>4.0f}|{x[1].ratio:>5.2f}%) {x[1].oscillation*100:>3.0f}', 
              'value': x[1].fpId} for x in labels.sort_values(['aerodromeOfDeparture','ratio']).iterrows()],)


@app.callback(
    [Output(component_id='my-slider', component_property='marks'),
     Output(component_id='my-slider', component_property='max'),
     Output(component_id='my-slider', component_property='value')],
    [Input(component_id='dd_trayectoria', component_property='value')],
    [State(component_id='sel_fecha', component_property='date')]
)
def actualizar_slider(trajectory, fecha):
    df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
    df = df[df.fpId == trajectory]
    
    if df.shape[0] != 0:
        marks = {x:str(x) for x in range(0,int(df.distance_dst.max()+100),100)}
        max_value = max(marks)
    else:
        marks = {0:'0',100:'100'}
        max_value = 100
    
    value = [0, max_value]
    
    return (marks, max_value, value)


@app.callback(
    [Output(component_id='inicial', component_property='children'),
     Output(component_id='reordenado', component_property='children'),
     Output(component_id='trayectory_description', component_property='children')],
    [Input(component_id='dd_trayectoria', component_property='value'),
     Input(component_id='my-slider', component_property='value'),
     Input(component_id='color_scale_dd', component_property='value')],
    [State(component_id='sel_fecha', component_property='date')]
)
def actualizar_grafico_perfiles(trajectory, rango, color_scale, fecha):
    df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
#     df = df.iloc[(df.shape[0]*rango[0])//100:(df.shape[0]*rango[1])//100]

    df = df[df.fpId == trajectory]
    
    max_distance = df.distance_dst.max()
    # print(df.distance_dst.max(), ((100-rango[0])*df.distance_dst.max())//100, ((100-rango[1])*(df.distance_dst.max()))//100)
    # df = df[(df.distance_dst < ((100-rango[0])*df.distance_dst.max())//100) &
    #         (df.distance_dst > ((100-rango[1])*df.distance_dst.max())//100)]
    df = df[df.distance_dst.between(rango[0],rango[1])]
    
    fig_inicial = px.scatter_mapbox(df, 
                        lat='latitude', lon='longitude',height=740,#width=900, 
                mapbox_style='open-street-map',zoom=4,
                hover_data=['timestamp','ordenFinal'], 
                color=color_scale, title='Orden inicial'
                 )
    
    fig_reordenado= px.scatter_mapbox(df, 
                        lat='latitude', lon='longitude',height=740,#width=900, 
                mapbox_style='open-street-map', zoom=4,
                hover_data=['ordenInicial','ordenFinal',
                            'timestamp', 'track', 'ground', 
                            'distance_org', 'distance_dst', 'fixed_timestamp', 'aerodromeOfDeparture'
                           ], 
                color='ordenFinal', title='Orden actual'
                 )
    props = dict(
        l=5,
        r=5,
        b=100,
        t=50,
        pad=0
    )
    fig_inicial.update_layout(coloraxis=dict(colorbar=dict(orientation='h', y=-0.15)), margin=props)
    fig_reordenado.update_layout(coloraxis=dict(colorbar=dict(orientation='h', y=-0.15)), margin=props)
    
    if trajectory:
        contenido = (
            # html.H5('Datos generales de la trayectoria'),
            html.Table(children=[
    #             html.Thead(children=[
    #                 html.Td('Datos del vuelo', colSpan=4, style=dict(textAlign='center', fontWeight='bold'))
    #             ]),
                html.Tbody(children=[
                    html.Tr(children=[
                        html.Td('Trayectoria:'),
                        html.Td(trajectory),
                        html.Td('Origen:'),
                        html.Td(df.iloc[0].aerodromeOfDeparture),
                        html.Td('Distancia inicial:'),
                        html.Td(f'{data[data.fpId == trajectory].initial.values[0]:>.2f}'),
                    ]),
                    html.Tr(children=[
                        html.Td('Rotación:'),
                        html.Td(f'{data[data.fpId == trajectory].rotation.values[0]:>.0f}'),
                        html.Td('Variación:'),
                        html.Td(f'{data[data.fpId == trajectory].ratio.values[0]:>.2f}%'),
                        html.Td('Distancia final:'),
                        html.Td(f'{data[data.fpId == trajectory].final.values[0]:>.2f}')
                    ]),

                ])
            ], style=dict(marginLeft='auto',marginRight='auto', marginTop=10)))
    else:
        contenido = []
    
    return ([dcc.Graph(id='grafico_inicial', figure=fig_inicial)], 
            [dcc.Graph(id='grafico_reordenado', figure=fig_reordenado)], 
            contenido)

In [ ]:
app.run_server(debug = True)
# mode='inline'